# Database Design Crash Course for Data Science Assessments

**Last Updated:** 25 January 2026

This notebook covers database design concepts commonly tested in data science interviews. We focus on normalisation, ER modelling, schema design patterns, and access control - going beyond just writing queries.

## Table of Contents

1. [Introduction and Setup](#1-introduction-and-setup)
2. [Database Design Fundamentals](#2-database-design-fundamentals)
3. [Normalisation](#3-normalisation)
4. [Entity-Relationship (ER) Modelling](#4-entity-relationship-er-modelling)
5. [Keys and Constraints](#5-keys-and-constraints)
6. [Data Warehouse Schemas](#6-data-warehouse-schemas)
7. [Indexing Strategies](#7-indexing-strategies)
8. [Access Control and Permissions](#8-access-control-and-permissions)
9. [Query Optimisation Basics](#9-query-optimisation-basics)
10. [Denormalisation](#10-denormalisation)
11. [Practice Questions](#11-practice-questions)
12. [Summary](#12-summary)

---

## 1. Introduction and Setup

**Database design** determines how data is structured, stored, and accessed. Good design ensures data integrity, reduces redundancy, and optimises performance.

**Why Database Design Matters for Data Scientists:**
- Understanding source data structure
- Designing analytical databases
- Optimising query performance
- Ensuring data quality

In [1]:
import sqlite3
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

print("All imports successful!")

All imports successful!


In [2]:
def run_query(conn: sqlite3.Connection, query: str) -> pd.DataFrame:
    """Execute SQL query and return results as DataFrame.
    
    Args:
        conn: SQLite connection.
        query: SQL query string.
    
    Returns:
        Query results as DataFrame.
    """
    return pd.read_sql_query(query, conn)


def execute_sql(conn: sqlite3.Connection, sql: str) -> None:
    """Execute SQL statement (CREATE, INSERT, etc.).
    
    Args:
        conn: SQLite connection.
        sql: SQL statement.
    """
    conn.executescript(sql)
    conn.commit()

---

## 2. Database Design Fundamentals

### OLTP vs OLAP

| Characteristic | OLTP | OLAP |
|---------------|------|------|
| Purpose | Day-to-day transactions | Analytics, reporting |
| Operations | INSERT, UPDATE, DELETE | SELECT (read-heavy) |
| Data | Current, detailed | Historical, aggregated |
| Design | Normalised (3NF) | Denormalised (star/snowflake) |
| Users | Many concurrent | Few analysts |

### Design Process

1. **Requirements gathering**: What data? What queries?
2. **Conceptual design**: ER diagrams
3. **Logical design**: Tables, relationships, normalisation
4. **Physical design**: Indexes, partitioning, storage

---

## 3. Normalisation

**Normalisation** organises tables to reduce redundancy and dependency. Each normal form builds on the previous.

### First Normal Form (1NF)

**Rules:**
- Each column contains atomic (indivisible) values
- Each column contains values of a single type
- Each row is unique (has a primary key)
- No repeating groups

In [3]:
print("VIOLATION of 1NF (repeating groups):")
unnormalised = pd.DataFrame({
    'order_id': [1, 2],
    'customer': ['Alice', 'Bob'],
    'products': ['Apple, Banana, Orange', 'Milk, Bread']  # Multiple values!
})
print(unnormalised)

print("\n1NF COMPLIANT (atomic values):")
normalised_1nf = pd.DataFrame({
    'order_id': [1, 1, 1, 2, 2],
    'customer': ['Alice', 'Alice', 'Alice', 'Bob', 'Bob'],
    'product': ['Apple', 'Banana', 'Orange', 'Milk', 'Bread']
})
print(normalised_1nf)

VIOLATION of 1NF (repeating groups):
   order_id customer               products
0         1    Alice  Apple, Banana, Orange
1         2      Bob            Milk, Bread

1NF COMPLIANT (atomic values):
   order_id customer product
0         1    Alice   Apple
1         1    Alice  Banana
2         1    Alice  Orange
3         2      Bob    Milk
4         2      Bob   Bread


### Second Normal Form (2NF)

**Rules:**
- Must be in 1NF
- All non-key attributes depend on the **entire** primary key (no partial dependencies)

Only applies to tables with composite primary keys.

In [4]:
print("VIOLATION of 2NF (partial dependency):")
print("Primary key: (student_id, course_id)")
print("student_name depends only on student_id, not the full key!")

violates_2nf = pd.DataFrame({
    'student_id': [1, 1, 2],
    'course_id': ['CS101', 'CS102', 'CS101'],
    'student_name': ['Alice', 'Alice', 'Bob'],  # Depends only on student_id
    'grade': ['A', 'B', 'A']
})
print(violates_2nf)

print("\n2NF COMPLIANT (separate tables):")
print("\nStudents table:")
students = pd.DataFrame({
    'student_id': [1, 2],
    'student_name': ['Alice', 'Bob']
})
print(students)

print("\nEnrollments table:")
enrollments = pd.DataFrame({
    'student_id': [1, 1, 2],
    'course_id': ['CS101', 'CS102', 'CS101'],
    'grade': ['A', 'B', 'A']
})
print(enrollments)

VIOLATION of 2NF (partial dependency):
Primary key: (student_id, course_id)
student_name depends only on student_id, not the full key!
   student_id course_id student_name grade
0           1     CS101        Alice     A
1           1     CS102        Alice     B
2           2     CS101          Bob     A

2NF COMPLIANT (separate tables):

Students table:
   student_id student_name
0           1        Alice
1           2          Bob

Enrollments table:
   student_id course_id grade
0           1     CS101     A
1           1     CS102     B
2           2     CS101     A


### Third Normal Form (3NF)

**Rules:**
- Must be in 2NF
- No transitive dependencies (non-key attribute depending on another non-key attribute)

In [5]:
print("VIOLATION of 3NF (transitive dependency):")
print("department_name depends on department_id, not directly on employee_id")

violates_3nf = pd.DataFrame({
    'employee_id': [1, 2, 3],
    'employee_name': ['Alice', 'Bob', 'Charlie'],
    'department_id': [10, 20, 10],
    'department_name': ['Sales', 'Engineering', 'Sales']  # Transitive!
})
print(violates_3nf)

print("\n3NF COMPLIANT (separate tables):")
print("\nEmployees table:")
employees = pd.DataFrame({
    'employee_id': [1, 2, 3],
    'employee_name': ['Alice', 'Bob', 'Charlie'],
    'department_id': [10, 20, 10]
})
print(employees)

print("\nDepartments table:")
departments = pd.DataFrame({
    'department_id': [10, 20],
    'department_name': ['Sales', 'Engineering']
})
print(departments)

VIOLATION of 3NF (transitive dependency):
department_name depends on department_id, not directly on employee_id
   employee_id employee_name  department_id department_name
0            1         Alice             10           Sales
1            2           Bob             20     Engineering
2            3       Charlie             10           Sales

3NF COMPLIANT (separate tables):

Employees table:
   employee_id employee_name  department_id
0            1         Alice             10
1            2           Bob             20
2            3       Charlie             10

Departments table:
   department_id department_name
0             10           Sales
1             20     Engineering


### Normalisation Summary

| Normal Form | Rule | Problem Solved |
|-------------|------|----------------|
| 1NF | Atomic values, no repeating groups | Data in cells is indivisible |
| 2NF | No partial dependencies | All columns depend on full PK |
| 3NF | No transitive dependencies | Non-key columns don't depend on each other |

---

## 4. Entity-Relationship (ER) Modelling

**ER diagrams** visualise database structure before implementation.

### Components

| Symbol | Meaning |
|--------|--------|
| Rectangle | Entity (table) |
| Oval | Attribute (column) |
| Diamond | Relationship |
| Line | Connects entities |

### Relationship Types

| Type | Example | Implementation |
|------|---------|----------------|
| One-to-One (1:1) | Person-Passport | FK in either table |
| One-to-Many (1:N) | Department-Employees | FK in "many" table |
| Many-to-Many (M:N) | Students-Courses | Junction table |

In [6]:
conn = sqlite3.connect(':memory:')

one_to_many_sql = """
-- One-to-Many: Department -> Employees
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department_id INTEGER,
    FOREIGN KEY (department_id) REFERENCES departments(department_id)
);

INSERT INTO departments VALUES (1, 'Sales'), (2, 'Engineering');
INSERT INTO employees VALUES (1, 'Alice', 1), (2, 'Bob', 2), (3, 'Charlie', 1);
"""

execute_sql(conn, one_to_many_sql)

print("One-to-Many Example:")
print(run_query(conn, """
    SELECT e.name as employee, d.name as department
    FROM employees e
    JOIN departments d ON e.department_id = d.department_id
"""))

One-to-Many Example:
  employee   department
0    Alice        Sales
1      Bob  Engineering
2  Charlie        Sales


In [7]:
many_to_many_sql = """
-- Many-to-Many: Students <-> Courses (via Enrollments junction table)
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE courses (
    course_id TEXT PRIMARY KEY,
    title TEXT NOT NULL
);

-- Junction table for M:N relationship
CREATE TABLE enrollments (
    student_id INTEGER,
    course_id TEXT,
    grade TEXT,
    PRIMARY KEY (student_id, course_id),
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
);

INSERT INTO students VALUES (1, 'Alice'), (2, 'Bob');
INSERT INTO courses VALUES ('CS101', 'Intro to CS'), ('CS102', 'Data Structures');
INSERT INTO enrollments VALUES (1, 'CS101', 'A'), (1, 'CS102', 'B'), (2, 'CS101', 'A');
"""

execute_sql(conn, many_to_many_sql)

print("Many-to-Many Example (Junction Table):")
print(run_query(conn, """
    SELECT s.name as student, c.title as course, e.grade
    FROM enrollments e
    JOIN students s ON e.student_id = s.student_id
    JOIN courses c ON e.course_id = c.course_id
"""))

Many-to-Many Example (Junction Table):
  student           course grade
0   Alice      Intro to CS     A
1   Alice  Data Structures     B
2     Bob      Intro to CS     A


### Preference Table Example (From Assessment)

A factory may prefer a specific supplier for a specific component. This requires a junction table with three foreign keys.

In [8]:
preference_sql = """
-- Factory-Component-Supplier Preference
CREATE TABLE suppliers (
    id_supplier INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE components (
    id_component INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE factories (
    id_factory INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    country TEXT
);

-- Preference: (factory, component) -> preferred supplier
CREATE TABLE preference (
    id_factory INTEGER,
    id_component INTEGER,
    id_supplier INTEGER,
    PRIMARY KEY (id_factory, id_component),
    FOREIGN KEY (id_factory) REFERENCES factories(id_factory),
    FOREIGN KEY (id_component) REFERENCES components(id_component),
    FOREIGN KEY (id_supplier) REFERENCES suppliers(id_supplier)
);

INSERT INTO suppliers VALUES (1, 'SupplierA'), (2, 'SupplierB');
INSERT INTO components VALUES (1, 'Motor'), (2, 'Battery');
INSERT INTO factories VALUES (1, 'Factory UK', 'UK'), (2, 'Factory US', 'USA');
INSERT INTO preference VALUES (1, 1, 1), (1, 2, 2), (2, 1, 2);
"""

execute_sql(conn, preference_sql)

print("Preference Table (Factory prefers Supplier for Component):")
print(run_query(conn, """
    SELECT f.name as factory, c.name as component, s.name as preferred_supplier
    FROM preference p
    JOIN factories f ON p.id_factory = f.id_factory
    JOIN components c ON p.id_component = c.id_component
    JOIN suppliers s ON p.id_supplier = s.id_supplier
"""))

Preference Table (Factory prefers Supplier for Component):
      factory component preferred_supplier
0  Factory UK     Motor          SupplierA
1  Factory UK   Battery          SupplierB
2  Factory US     Motor          SupplierB


---

## 5. Keys and Constraints

### Types of Keys

| Key Type | Description | Example |
|----------|-------------|--------|
| Primary Key (PK) | Unique identifier for each row | employee_id |
| Foreign Key (FK) | References PK in another table | department_id in employees |
| Candidate Key | Could be PK (unique, non-null) | email, SSN |
| Composite Key | PK made of multiple columns | (student_id, course_id) |
| Surrogate Key | Artificial PK (auto-increment) | id |
| Natural Key | Real-world identifier | ISBN, email |

### Constraints

| Constraint | Purpose | SQL |
|------------|---------|-----|
| NOT NULL | Column cannot be NULL | `name TEXT NOT NULL` |
| UNIQUE | All values must be different | `email TEXT UNIQUE` |
| PRIMARY KEY | NOT NULL + UNIQUE | `id INTEGER PRIMARY KEY` |
| FOREIGN KEY | Referential integrity | `FOREIGN KEY (dept_id) REFERENCES dept(id)` |
| CHECK | Custom validation | `CHECK (age >= 0)` |
| DEFAULT | Default value | `status TEXT DEFAULT 'active'` |

In [9]:
constraints_sql = """
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    sku TEXT UNIQUE,
    price REAL CHECK (price >= 0),
    quantity INTEGER DEFAULT 0,
    category_id INTEGER,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""

execute_sql(conn, constraints_sql)

print("Table created with constraints:")
print(run_query(conn, "PRAGMA table_info(products)"))

Table created with constraints:
   cid         name     type  notnull         dflt_value  pk
0    0   product_id  INTEGER        0                NaN   1
1    1         name     TEXT        1                NaN   0
2    2          sku     TEXT        0                NaN   0
3    3        price     REAL        0                NaN   0
4    4     quantity  INTEGER        0                  0   0
5    5  category_id  INTEGER        0                NaN   0
6    6   created_at     TEXT        0  CURRENT_TIMESTAMP   0


---

## 6. Data Warehouse Schemas

Data warehouses use denormalised schemas for analytical queries.

### Star Schema

**Structure:**
- Central **fact table** (measurements, metrics)
- Surrounding **dimension tables** (descriptive attributes)

**Advantages:** Simple queries, fast reads

**Disadvantages:** Data redundancy in dimensions

In [10]:
star_schema_sql = """
-- DIMENSION TABLES
CREATE TABLE dim_date (
    date_key INTEGER PRIMARY KEY,
    full_date TEXT,
    day_of_week TEXT,
    month TEXT,
    quarter INTEGER,
    year INTEGER
);

CREATE TABLE dim_product (
    product_key INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    brand TEXT
);

CREATE TABLE dim_store (
    store_key INTEGER PRIMARY KEY,
    store_name TEXT,
    city TEXT,
    region TEXT
);

-- FACT TABLE (centre of the star)
CREATE TABLE fact_sales (
    sale_id INTEGER PRIMARY KEY,
    date_key INTEGER,
    product_key INTEGER,
    store_key INTEGER,
    quantity INTEGER,
    revenue REAL,
    FOREIGN KEY (date_key) REFERENCES dim_date(date_key),
    FOREIGN KEY (product_key) REFERENCES dim_product(product_key),
    FOREIGN KEY (store_key) REFERENCES dim_store(store_key)
);

-- Insert sample data
INSERT INTO dim_date VALUES 
    (20240101, '2024-01-01', 'Monday', 'January', 1, 2024),
    (20240102, '2024-01-02', 'Tuesday', 'January', 1, 2024);

INSERT INTO dim_product VALUES 
    (1, 'Laptop', 'Electronics', 'TechBrand'),
    (2, 'Phone', 'Electronics', 'MobileCo');

INSERT INTO dim_store VALUES 
    (1, 'Downtown', 'London', 'South'),
    (2, 'Mall', 'Manchester', 'North');

INSERT INTO fact_sales VALUES 
    (1, 20240101, 1, 1, 5, 5000),
    (2, 20240101, 2, 1, 10, 8000),
    (3, 20240102, 1, 2, 3, 3000);
"""

execute_sql(conn, star_schema_sql)

print("Star Schema Query - Sales by Product and Region:")
print(run_query(conn, """
    SELECT 
        p.category,
        s.region,
        SUM(f.quantity) as total_quantity,
        SUM(f.revenue) as total_revenue
    FROM fact_sales f
    JOIN dim_product p ON f.product_key = p.product_key
    JOIN dim_store s ON f.store_key = s.store_key
    GROUP BY p.category, s.region
"""))

Star Schema Query - Sales by Product and Region:
      category region  total_quantity  total_revenue
0  Electronics  North               3         3000.0
1  Electronics  South              15        13000.0


### Snowflake Schema

**Structure:** Like star schema but dimensions are normalised into sub-dimensions.

**Example:** Instead of storing `category` in dim_product, create a separate dim_category table.

| Aspect | Star Schema | Snowflake Schema |
|--------|-------------|------------------|
| Dimension tables | Denormalised | Normalised |
| Query complexity | Simpler | More joins |
| Storage | More redundancy | Less redundancy |
| Query speed | Faster | Slower |

In [11]:
print("Star vs Snowflake Schema Comparison:")
print()
print("STAR SCHEMA (dimension denormalised):")
print("  dim_product: product_key, name, category, brand")
print()
print("SNOWFLAKE SCHEMA (dimension normalised):")
print("  dim_product: product_key, name, category_key, brand_key")
print("  dim_category: category_key, category_name")
print("  dim_brand: brand_key, brand_name")

Star vs Snowflake Schema Comparison:

STAR SCHEMA (dimension denormalised):
  dim_product: product_key, name, category, brand

SNOWFLAKE SCHEMA (dimension normalised):
  dim_product: product_key, name, category_key, brand_key
  dim_category: category_key, category_name
  dim_brand: brand_key, brand_name


---

## 7. Indexing Strategies

**Indexes** speed up data retrieval at the cost of slower writes and extra storage.

### Index Types

| Type | Description | Use Case |
|------|-------------|----------|
| B-tree | Balanced tree, default | Equality and range queries |
| Hash | Hash table | Equality only |
| Bitmap | Bit arrays | Low cardinality columns |
| Composite | Multiple columns | Multi-column WHERE |

### When to Create Indexes

**DO index:**
- Primary keys (automatic)
- Foreign keys
- Columns in WHERE clauses
- Columns in JOIN conditions
- Columns in ORDER BY

**DON'T index:**
- Small tables
- Columns with low selectivity
- Tables with heavy INSERT/UPDATE

In [12]:
index_sql = """
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    status TEXT,
    total REAL
);

-- Index on foreign key (common join)
CREATE INDEX idx_orders_customer ON orders(customer_id);

-- Index on frequently filtered column
CREATE INDEX idx_orders_date ON orders(order_date);

-- Composite index for common query pattern
CREATE INDEX idx_orders_customer_date ON orders(customer_id, order_date);
"""

execute_sql(conn, index_sql)

print("Indexes created on orders table:")
print(run_query(conn, "PRAGMA index_list(orders)"))

Indexes created on orders table:
   seq                      name  unique origin  partial
0    0  idx_orders_customer_date       0      c        0
1    1           idx_orders_date       0      c        0
2    2       idx_orders_customer       0      c        0


In [13]:
print("Index Selection Guidelines:")
print()
print("Query: SELECT * FROM orders WHERE customer_id = 100")
print("  -> Use index on customer_id")
print()
print("Query: SELECT * FROM orders WHERE customer_id = 100 AND order_date > '2024-01-01'")
print("  -> Use composite index on (customer_id, order_date)")
print()
print("Query: SELECT * FROM orders ORDER BY order_date DESC LIMIT 10")
print("  -> Use index on order_date")

Index Selection Guidelines:

Query: SELECT * FROM orders WHERE customer_id = 100
  -> Use index on customer_id

Query: SELECT * FROM orders WHERE customer_id = 100 AND order_date > '2024-01-01'
  -> Use composite index on (customer_id, order_date)

Query: SELECT * FROM orders ORDER BY order_date DESC LIMIT 10
  -> Use index on order_date


---

## 8. Access Control and Permissions

**Access control** restricts who can see or modify data.

### GRANT and REVOKE

```sql
-- Grant permissions
GRANT SELECT ON table_name TO user_name;
GRANT INSERT, UPDATE ON table_name TO user_name;
GRANT ALL PRIVILEGES ON table_name TO user_name;

-- Revoke permissions
REVOKE SELECT ON table_name FROM user_name;

-- Column-level permissions (important for PII protection)
GRANT SELECT (id, town, buy_date) ON customers TO statistics;
```

### Permission Types

| Permission | Allows |
|------------|--------|
| SELECT | Read data |
| INSERT | Add new rows |
| UPDATE | Modify existing rows |
| DELETE | Remove rows |
| ALL | All permissions |

In [14]:
print("Column-Level Permissions Example (From Assessment):")
print()
print("Table: customers")
print("  - id (safe)")
print("  - address (PII - sensitive!)")
print("  - town (safe)")
print("  - buy_date (safe)")
print("  - purchase_plan (safe)")
print()
print("Requirement: 'statistics' role needs read access but NOT to address")
print()
print("Solution (column-level GRANT):")
print("")
print("  GRANT SELECT (id, town, buy_date, purchase_plan)")
print("  ON customers")
print("  TO statistics;")
print()

Column-Level Permissions Example (From Assessment):

Table: customers
  - id (safe)
  - address (PII - sensitive!)
  - town (safe)
  - buy_date (safe)
  - purchase_plan (safe)

Requirement: 'statistics' role needs read access but NOT to address

Solution (column-level GRANT):

  GRANT SELECT (id, town, buy_date, purchase_plan)
  ON customers
  TO statistics;



### Views for Access Control

**Views** can provide a security layer by exposing only certain columns or rows.

In [15]:
view_sql = """
CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    address TEXT,  -- PII
    town TEXT,
    email TEXT  -- PII
);

INSERT INTO customers VALUES 
    (1, 'Alice', '123 Main St', 'London', 'alice@email.com'),
    (2, 'Bob', '456 Oak Ave', 'Manchester', 'bob@email.com');

-- View that hides PII
CREATE VIEW customers_safe AS
SELECT id, name, town
FROM customers;
"""

execute_sql(conn, view_sql)

print("Full table (includes PII):")
print(run_query(conn, "SELECT * FROM customers"))

print("\nSafe view (PII hidden):")
print(run_query(conn, "SELECT * FROM customers_safe"))

Full table (includes PII):
   id   name      address        town            email
0   1  Alice  123 Main St      London  alice@email.com
1   2    Bob  456 Oak Ave  Manchester    bob@email.com

Safe view (PII hidden):
   id   name        town
0   1  Alice      London
1   2    Bob  Manchester


---

## 9. Query Optimisation Basics

### Reading Execution Plans

Use `EXPLAIN` (or `EXPLAIN ANALYZE`) to understand how queries execute.

### Common Optimisation Techniques

| Technique | When to Use |
|-----------|-------------|
| Add indexes | Slow WHERE/JOIN/ORDER BY |
| Aggregate early | Reduce data before joins |
| Use covering indexes | Avoid table lookups |
| Avoid SELECT * | Only fetch needed columns |
| Use EXISTS vs IN | Subquery optimisation |

In [16]:
print("Query Optimisation Example (From Assessment):")
print()
print("Question: Most efficient way to count products per customer?")
print()
print("INEFFICIENT (join first, then aggregate):")
print("""
SELECT customer_id, COUNT(*)
FROM purchase_order po
JOIN order_product op ON po.order_id = op.order_id
GROUP BY customer_id;
""")
print()
print("EFFICIENT (aggregate early):")
print("""
SELECT po.customer_id, SUM(product_count)
FROM purchase_order po
JOIN (
    SELECT order_id, COUNT(*) AS product_count
    FROM order_product
    GROUP BY order_id
) op ON po.order_id = op.order_id
GROUP BY po.customer_id;
""")
print("Why better: Aggregation reduces rows BEFORE the join.")

Query Optimisation Example (From Assessment):

Question: Most efficient way to count products per customer?

INEFFICIENT (join first, then aggregate):

SELECT customer_id, COUNT(*)
FROM purchase_order po
JOIN order_product op ON po.order_id = op.order_id
GROUP BY customer_id;


EFFICIENT (aggregate early):

SELECT po.customer_id, SUM(product_count)
FROM purchase_order po
JOIN (
    SELECT order_id, COUNT(*) AS product_count
    FROM order_product
    GROUP BY order_id
) op ON po.order_id = op.order_id
GROUP BY po.customer_id;

Why better: Aggregation reduces rows BEFORE the join.


In [17]:
print("Execution Plan (SQLite EXPLAIN):")
execute_sql(conn, """
    CREATE TABLE test_orders (id INTEGER PRIMARY KEY, customer_id INTEGER);
    CREATE INDEX idx_test_customer ON test_orders(customer_id);
""")

print(run_query(conn, "EXPLAIN QUERY PLAN SELECT * FROM test_orders WHERE customer_id = 100"))

Execution Plan (SQLite EXPLAIN):
   id  parent  notused                                             detail
0   2       0        0  SEARCH test_orders USING COVERING INDEX idx_te...


---

## 10. Denormalisation

**Denormalisation** intentionally adds redundancy to improve read performance.

### When to Denormalise

| Scenario | Reason |
|----------|--------|
| Read-heavy workloads | Fewer joins = faster reads |
| Data warehouses | Analytics need wide tables |
| Caching calculated values | Avoid repeated calculations |
| Reporting tables | Pre-aggregated for dashboards |

### Denormalisation Techniques

1. **Pre-computed columns**: Store derived values
2. **Duplicate columns**: Copy FK values to avoid joins
3. **Summary tables**: Store aggregations
4. **Materialised views**: Cached query results

In [18]:
print("Normalised vs Denormalised Comparison:")
print()
print("NORMALISED (3NF):")
print("  orders: order_id, customer_id, order_date")
print("  customers: customer_id, name, email")
print("  Query: SELECT o.*, c.name FROM orders o JOIN customers c ...")
print()
print("DENORMALISED:")
print("  orders: order_id, customer_id, customer_name, order_date")
print("  Query: SELECT * FROM orders (no join needed!)")
print()
print("Trade-off: Faster reads, but customer_name must be updated")
print("           in multiple places if it changes.")

Normalised vs Denormalised Comparison:

NORMALISED (3NF):
  orders: order_id, customer_id, order_date
  customers: customer_id, name, email
  Query: SELECT o.*, c.name FROM orders o JOIN customers c ...

DENORMALISED:
  orders: order_id, customer_id, customer_name, order_date
  Query: SELECT * FROM orders (no join needed!)

Trade-off: Faster reads, but customer_name must be updated
           in multiple places if it changes.


## 12. Practice Questions

Test your understanding with these interview-style questions. Try to solve each question in the empty code cell before revealing the answer.

### Question 1: Identify Normal Form Violation

Which normal form does this table violate, and how would you fix it?

```
orders(order_id, customer_id, customer_name, customer_email, product, quantity)
```

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**Violation:** 2NF (partial dependency) and 3NF (transitive dependency)

- `customer_name` and `customer_email` depend only on `customer_id`, not the full key
- These are transitive dependencies through `customer_id`

**Fix:**
```sql
-- Customers table
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT,
    customer_email TEXT
);

-- Orders table (normalised)
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER REFERENCES customers(customer_id),
    product TEXT,
    quantity INTEGER
);
```

</details>


---

### Question 2: Design a Junction Table

Design a schema for a system where authors can write multiple books, and books can have multiple authors.

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

```sql
CREATE TABLE authors (
    author_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE
);

CREATE TABLE books (
    book_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    isbn TEXT UNIQUE,
    publication_year INTEGER
);

-- Junction table for M:N relationship
CREATE TABLE book_authors (
    book_id INTEGER,
    author_id INTEGER,
    author_order INTEGER,  -- e.g., first author, second author
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
);
```

</details>


---

### Question 3: Star Schema Design

Design a star schema for tracking website page views with dimensions for date, page, and user.

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

```sql
-- Dimension: Date
CREATE TABLE dim_date (
    date_key INTEGER PRIMARY KEY,
    full_date DATE,
    day_of_week TEXT,
    month TEXT,
    quarter INTEGER,
    year INTEGER,
    is_weekend BOOLEAN
);

-- Dimension: Page
CREATE TABLE dim_page (
    page_key INTEGER PRIMARY KEY,
    url TEXT,
    page_title TEXT,
    section TEXT,
    content_type TEXT
);

-- Dimension: User
CREATE TABLE dim_user (
    user_key INTEGER PRIMARY KEY,
    user_id TEXT,
    user_type TEXT,  -- anonymous, registered, premium
    country TEXT,
    device_type TEXT
);

-- Fact: Page Views
CREATE TABLE fact_page_views (
    view_id INTEGER PRIMARY KEY,
    date_key INTEGER REFERENCES dim_date(date_key),
    page_key INTEGER REFERENCES dim_page(page_key),
    user_key INTEGER REFERENCES dim_user(user_key),
    view_count INTEGER,
    time_on_page_seconds INTEGER,
    bounce BOOLEAN
);
```

</details>


---

### Question 4: Index Selection

Given these queries, what indexes would you create?

```sql
SELECT * FROM orders WHERE customer_id = ? AND status = 'pending';
SELECT * FROM orders WHERE order_date BETWEEN ? AND ?;
SELECT customer_id, COUNT(*) FROM orders GROUP BY customer_id;
```

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

```sql
-- Query 1: Composite index on (customer_id, status)
CREATE INDEX idx_orders_customer_status ON orders(customer_id, status);

-- Query 2: Index on order_date for range query
CREATE INDEX idx_orders_date ON orders(order_date);

-- Query 3: Index on customer_id for GROUP BY
-- (Already covered by idx_orders_customer_status if customer_id is first)
```

**Note:** The composite index `(customer_id, status)` also helps Query 3 because the leading column is `customer_id`.

</details>


---

### Question 5: Column-Level Permissions

Write SQL to grant a `reporting` role read access to a `transactions` table, but exclude the `credit_card_number` column.

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

```sql
-- Assuming table structure:
-- transactions(id, customer_id, amount, credit_card_number, transaction_date)

-- Method 1: Column-level GRANT (if supported by DBMS)
GRANT SELECT (id, customer_id, amount, transaction_date)
ON transactions
TO reporting;

-- Method 2: Create a view (works in all DBMS)
CREATE VIEW transactions_safe AS
SELECT id, customer_id, amount, transaction_date
FROM transactions;

GRANT SELECT ON transactions_safe TO reporting;
```

</details>


---

### Question 6: Identify Relationship Type

For each scenario, identify the relationship type (1:1, 1:N, M:N) and how to implement it:

a) Person and Passport
b) Department and Employees
c) Products and Tags

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**a) Person-Passport: One-to-One (1:1)**
```sql
CREATE TABLE persons (person_id PRIMARY KEY, name);
CREATE TABLE passports (
    passport_id PRIMARY KEY,
    person_id UNIQUE REFERENCES persons(person_id),  -- UNIQUE enforces 1:1
    passport_number
);
```

**b) Department-Employees: One-to-Many (1:N)**
```sql
CREATE TABLE departments (dept_id PRIMARY KEY, name);
CREATE TABLE employees (
    emp_id PRIMARY KEY,
    name,
    dept_id REFERENCES departments(dept_id)  -- FK in "many" side
);
```

**c) Products-Tags: Many-to-Many (M:N)**
```sql
CREATE TABLE products (product_id PRIMARY KEY, name);
CREATE TABLE tags (tag_id PRIMARY KEY, name);
CREATE TABLE product_tags (  -- Junction table
    product_id REFERENCES products(product_id),
    tag_id REFERENCES tags(tag_id),
    PRIMARY KEY (product_id, tag_id)
);
```

</details>


---

### Question 7: Denormalisation Decision

Should you denormalise in these scenarios? Explain.

a) E-commerce product catalogue (read-heavy)
b) Banking transaction system (write-heavy, needs audit)
c) Real-time analytics dashboard

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**a) E-commerce product catalogue: YES, denormalise**
- Read-heavy workload (users browsing)
- Products don't change frequently
- Store category_name with product to avoid joins

**b) Banking transaction system: NO, keep normalised**
- Write-heavy with strict consistency needs
- Audit requirements need single source of truth
- Data integrity is critical

**c) Real-time analytics dashboard: YES, denormalise**
- Create materialised views or summary tables
- Pre-aggregate data for fast dashboard rendering
- Accept some data staleness for speed

</details>


---

### Question 8: Fix the Schema

This schema has issues. Identify and fix them:

```sql
CREATE TABLE users (
    user_id,
    name,
    phone1,
    phone2,
    phone3,
    address
);
```

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**Issues:**
1. No data types specified
2. No PRIMARY KEY defined
3. Violates 1NF: repeating groups (phone1, phone2, phone3)
4. Address should probably be broken into components

**Fixed schema:**
```sql
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    street_address TEXT,
    city TEXT,
    postcode TEXT,
    country TEXT
);

-- Separate table for phones (1:N relationship)
CREATE TABLE user_phones (
    phone_id INTEGER PRIMARY KEY,
    user_id INTEGER REFERENCES users(user_id),
    phone_number TEXT NOT NULL,
    phone_type TEXT  -- 'mobile', 'home', 'work'
);
```

</details>


---

### Question 9: Surrogate vs Natural Keys

When would you use a surrogate key (auto-increment ID) vs a natural key (like email or ISBN)?

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**Use Surrogate Key when:**
- Natural key might change (email can change)
- Natural key is long (URLs, full names)
- Natural key is composite (multiple columns)
- Want consistent key format across tables
- Performance matters (integers are faster)

**Use Natural Key when:**
- Key is stable and never changes (ISBN, SSN)
- Key is meaningful and used in queries
- Data comes from external source with its own ID
- Want to prevent duplicate records naturally

**Best Practice:** Often use both - surrogate key as PK, natural key as UNIQUE constraint.

```sql
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY,  -- Surrogate
    isbn TEXT UNIQUE NOT NULL,    -- Natural (unique constraint)
    title TEXT
);
```

</details>


---

### Question 10: Query Optimisation

Optimise this query:

```sql
SELECT c.name, COUNT(o.order_id) as order_count, SUM(o.total) as total_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC;
```

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**Issues:**
1. LEFT JOIN with WHERE on right table acts like INNER JOIN
2. Missing index on order_date and customer_id
3. Could filter orders first, then join

**Optimised:**
```sql
-- Create indexes
CREATE INDEX idx_orders_date_customer ON orders(order_date, customer_id);

-- Optimised query (filter first)
SELECT c.name, 
       COALESCE(o.order_count, 0) as order_count,
       COALESCE(o.total_spent, 0) as total_spent
FROM customers c
LEFT JOIN (
    SELECT customer_id,
           COUNT(*) as order_count,
           SUM(total) as total_spent
    FROM orders
    WHERE order_date >= '2024-01-01'
    GROUP BY customer_id
) o ON c.customer_id = o.customer_id
ORDER BY total_spent DESC;
```

**Why better:**
- Aggregation happens on filtered subset first
- LEFT JOIN preserved (shows customers with 0 orders)
- Index supports the WHERE and GROUP BY

</details>


---

### Question 11: ACID Properties

Explain each ACID property and give an example of what could go wrong without it.

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

**A - Atomicity:** All or nothing. Transaction fully completes or fully rolls back.
- Without: Transfer $100 from A to B could debit A but not credit B (money disappears!)

**C - Consistency:** Database moves from one valid state to another.
- Without: Could violate constraints (negative balance, invalid foreign key)

**I - Isolation:** Concurrent transactions don't interfere with each other.
- Without: Two withdrawals from same account could both succeed when only one should

**D - Durability:** Committed transactions survive system failures.
- Without: Power outage could lose committed transactions

</details>


---

### Question 12: Design a Preference Table

Design a table to store: "A factory prefers a specific supplier for a specific component for all motors it produces."

In [ ]:
# Write your solution here

<details>
<summary>Click to reveal answer</summary>

```sql
-- Existing tables
CREATE TABLE factories (id_factory INTEGER PRIMARY KEY, name TEXT);
CREATE TABLE suppliers (id_supplier INTEGER PRIMARY KEY, name TEXT);
CREATE TABLE components (id_component INTEGER PRIMARY KEY, name TEXT);

-- Preference table: (factory, component) -> supplier
CREATE TABLE preference (
    id_factory INTEGER,
    id_component INTEGER,
    id_supplier INTEGER,
    PRIMARY KEY (id_factory, id_component),  -- Each factory has ONE supplier per component
    FOREIGN KEY (id_factory) REFERENCES factories(id_factory),
    FOREIGN KEY (id_component) REFERENCES components(id_component),
    FOREIGN KEY (id_supplier) REFERENCES suppliers(id_supplier)
);
```

**Key insight:** The primary key is `(id_factory, id_component)` because each factory-component pair maps to exactly one preferred supplier.

</details>


---

## 12. Summary

This notebook covered database design concepts:

1. **OLTP vs OLAP**: Transaction processing vs analytics
2. **Normalisation**: 1NF (atomic), 2NF (no partial deps), 3NF (no transitive deps)
3. **ER Modelling**: Entities, relationships (1:1, 1:N, M:N), junction tables
4. **Keys**: Primary, foreign, composite, surrogate vs natural
5. **Data Warehouse Schemas**: Star (fact + dimensions) vs snowflake
6. **Indexing**: B-tree, composite indexes, when to index
7. **Access Control**: GRANT, REVOKE, column-level permissions, views
8. **Query Optimisation**: Aggregate early, use indexes, execution plans
9. **Denormalisation**: When to break normal forms for performance

---

### Key Interview Tips

- **Know normal forms**: Be able to identify violations and fix them
- **Junction tables**: Essential for M:N relationships
- **Star schema**: Fact tables have metrics, dimension tables have attributes
- **Index trade-offs**: Faster reads, slower writes
- **Column-level security**: Use GRANT or views to hide sensitive data
- **Aggregate early**: Reduce data before joins for better performance

In [ ]:
conn.close()